In [8]:
!pip uninstall numpy
!pip install numpy==1.26.4


^C
^C


   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/15.5 MB 11.8 MB/s eta 0:00:02
   ------------------- -------------------- 7.6/15.5 MB 20.4 MB/s eta 0:00:01
   ------------------------------------- -- 14.4/15.5 MB 23.9 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibjpeg-libjpeg 2.3.0 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [10]:
import os
import pydicom
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split

# Función para crear un volumen 3D desde múltiples archivos DICOM
def create_3d_volume(dicom_files):
    slices = []
    spacings = []

    for dicom_path in dicom_files:
        try:
            dicom = pydicom.dcmread(dicom_path)
            image = dicom.pixel_array.astype(np.int16)

            # Normalizar valores HU
            image[image == -2000] = 0
            intercept = dicom.RescaleIntercept
            slope = dicom.RescaleSlope
            image = image * slope + intercept
            slices.append(image)

            # Extraer la resolución del voxel
            pixel_spacing = getattr(dicom, "PixelSpacing", [1, 1])
            slice_thickness = getattr(dicom, "SliceThickness", 1)
            spacings.append(list(pixel_spacing) + [slice_thickness])

        except PermissionError:
            print(f"No se pudo acceder al archivo: {dicom_path}. Verifica permisos.")
            continue
        except Exception as e:
            print(f"Error al procesar el archivo {dicom_path}: {e}")
            continue

    # Ordenar los cortes por su posición en Z (si hay un atributo SliceLocation)
    slices = np.array(slices)
    spacings = np.mean(spacings, axis=0) if spacings else [1, 1, 1]  # Espaciado por defecto si no se encuentra
    return np.stack(slices, axis=0), spacings

# Función para cargar y preprocesar las imágenes y volúmenes
def cargar_y_preprocesar_imagenes(directorio, target_shape=(64, 64, 32)):
    imagenes = []
    etiquetas = []
    
    # Recorrer la carpeta fibrosis_reducido
    fibrosis_dir = os.path.join(directorio, 'fibrosis_reducido')
    print(f"Comprobando carpeta de fibrosis: {fibrosis_dir}")
    for root, dirs, files in os.walk(fibrosis_dir):
        dicom_files = [os.path.join(root, archivo) for archivo in files if archivo.endswith('.dcm')]
        if dicom_files:
            volumen, _ = create_3d_volume(dicom_files)
            volumen_redimensionado = resize(volumen, target_shape, mode='constant', anti_aliasing=True)
            imagenes.append(volumen_redimensionado)
            etiquetas.append(0)  # 0 para fibrosis
    
    print(f"Imágenes de fibrosis cargadas: {len(imagenes)}")
    
    # Recorrer la carpeta CT_cancer
    cancer_dir = os.path.join(directorio, 'CT_cancer')
    print(f"Comprobando carpeta de cáncer: {cancer_dir}")
    for root, dirs, files in os.walk(cancer_dir):
        dicom_files = [os.path.join(root, archivo) for archivo in files if archivo.endswith('.dcm')]
        if dicom_files:
            volumen, _ = create_3d_volume(dicom_files)
            volumen_redimensionado = resize(volumen, target_shape, mode='constant', anti_aliasing=True)
            imagenes.append(volumen_redimensionado)
            etiquetas.append(1)  # 1 para cáncer
    
    print(f"Imágenes de cáncer cargadas: {len(imagenes)}")
    
    # Verifica si hay imágenes cargadas
    print(f"Total de imágenes cargadas: {len(imagenes)}")
    
    # Convertir a arrays de numpy y añadir el canal de color
    imagenes = np.array(imagenes)[..., np.newaxis]  # Añadir canal para imágenes en escala de grises
    etiquetas = np.array(etiquetas)
    
    return imagenes, etiquetas

# Ruta al directorio donde están las carpetas fibrosis_reducido y CT_cancer
directorio_base = "C:\\Users\\Marcos\\reto-1-mas-alla-de-la-mirada-humana"

# Cargar y preprocesar las imágenes
imagenes, etiquetas = cargar_y_preprocesar_imagenes(directorio_base)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Verificar las dimensiones de los datos
print(f"Datos de entrenamiento: {X_train.shape}") 
print(f"Datos de prueba: {X_test.shape}")


Comprobando carpeta de fibrosis: C:\Users\Marcos\reto-1-mas-alla-de-la-mirada-humana\fibrosis_reducido
Error al procesar el archivo C:\Users\Marcos\reto-1-mas-alla-de-la-mirada-humana\fibrosis_reducido\ID00011637202177653955184\1.dcm: Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires gdcm>=3.0.10
	pylibjpeg - requires pylibjpeg>=2.0 and pylibjpeg-libjpeg>=2.1
Error al procesar el archivo C:\Users\Marcos\reto-1-mas-alla-de-la-mirada-humana\fibrosis_reducido\ID00011637202177653955184\10.dcm: Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires gdcm>=3.0.10
	pylibjpeg - requires pylibjpeg>=2.0 and pylibjpeg-libjpeg>=2.1
Error al procesar el archivo C:\Users\Marcos\reto-1-mas-alla-de-la-mirada-humana\fibrosis_reducido\ID00011

ValueError: need at least one array to stack

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Crear modelo de red 3D basado en la estructura de AlexNet
model = Sequential()

# Capa 1: Convolución 3D
model.add(Conv3D(96, (5, 5, 5), strides=(2, 2, 2), activation='relu', input_shape=(64, 64, 32, 1)))  # Suponemos que 1 es el canal (escala de grises)
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2)))

# Capa 2: Convolución 3D
model.add(Conv3D(256, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2)))

# Capa 3: Convolución 3D
model.add(Conv3D(384, (3, 3, 3), activation='relu'))

# Capa 4: Convolución 3D
model.add(Conv3D(384, (3, 3, 3), activation='relu'))

# Capa 5: Convolución 3D
model.add(Conv3D(256, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2)))

# Aplanar las salidas de las capas convolucionales
model.add(Flatten())

# Capa 6: Densa
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Capa 7: Densa
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Capa de salida: Clasificación binaria
model.add(Dense(1, activation='sigmoid'))  # Sigmoid para clasificación binaria

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()


Entrenamiento y Evaluación

In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2)

# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {test_acc * 100:.2f}%")


Generar Predicciones

In [ ]:
def predecir_imagen(model, ruta_imagen, target_shape=(64, 64, 32)):
    # Cargar el archivo DICOM
    archivo_dicom = pydicom.dcmread(ruta_imagen)
    imagen = archivo_dicom.pixel_array
    
    # Redimensionar la imagen 3D
    imagen_redimensionada = resize(imagen, target_shape, mode='constant', anti_aliasing=True)
    
    # Añadir el canal (escala de grises) y la dimensión del batch
    imagen_redimensionada = imagen_redimensionada[..., np.newaxis]
    imagen_redimensionada = np.expand_dims(imagen_redimensionada, axis=0)
    
    # Realizar la predicción
    prediccion = model.predict(imagen_redimensionada)
    probabilidad = prediccion[0][0]
    clase_predicha = 'Cáncer' if probabilidad > 0.5 else 'Fibrosis'
    
    return clase_predicha, probabilidad

# Ejemplo de uso
ruta_imagen = "C:\\Users\\Marcos\\reto-1-mas-alla-de-la-mirada-humana\\ID00407637202308788732304"
clase, probabilidad = predecir_imagen(model, ruta_imagen)
print(f"Predicción: {clase}, Probabilidad: {probabilidad:.4f}")
